In [ ]:
import pandas as pd
import tensorflow as tf
from models.utils.Preprocess import Preprocessor
from models.transformer.evaluate import Evaluate
from models.transformer.train import CustomSchedule, get_model

print(tf.__version__)

In [ ]:
train_data = pd.read_csv('../data/ChatbotData_merged.csv')
train_data = train_data.dropna()
train_data.head()

In [ ]:
print('챗봇 샘플의 개수 :', len(train_data))
print(train_data.isnull().sum())

In [ ]:
questions = train_data['Q'].to_list()
answers = train_data['A'].to_list()
sentence_list = questions + answers

preprocessor = Preprocessor()

In [ ]:
# 갑자기 왜 안되지?
from utils.load_tokenizer import load_tokenizer

tokenizer = load_tokenizer('vocab_32000.txt')
print(tokenizer)

In [ ]:
# tokenizer = preprocessor.get_tokenizer(sentence_list, target_vocab_size=32000)
# print(tokenizer)

In [ ]:
questions, answers = preprocessor.tokenize_and_filter(questions, answers, tokenizer)

print('질문 데이터의 크기(shape) :', questions.shape)
print('답변 데이터의 크기(shape) :', answers.shape)

# 0번 샘플을 임의로 출력
print(questions[0])
print(answers[0])

In [ ]:
dataset = preprocessor.get_train_dataset(questions, answers, batch_size=256, buffer_size=20000)

In [ ]:
NUM_LAYERS = 4
model = get_model(
    vocab_size=tokenizer.vocab_size+2,
    num_layers=NUM_LAYERS,
    dff=512,
    d_model=256,
    num_heads=8,
    dropout=0.2)


def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, preprocessor.MAX_LENGTH-1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)


def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, preprocessor.MAX_LENGTH-1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

learning_rate = CustomSchedule(d_model=256)


optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
'''No checkpoint'''
def train_model(epochs=50, mini_epoch=10, is_contiune=False):
    epoch_saved_path = f'./checkpoints/weights/vocab_{tokenizer.vocab_size}_layers_{NUM_LAYERS}_epoch_saved.txt'
    # 저장된 모델이 있는 경우
    if is_contiune:
        with open(epoch_saved_path, 'r', encoding='utf-8') as f:
            total_epoch = int(f.read())
        model_path = f'checkpoints/weights/transformer_weight_vocab_{tokenizer.vocab_size}_layers_{NUM_LAYERS}_{total_epoch}.h5'
        model.load_weights(model_path)
        print(f"{model_path.replace('checkpoints/weights/', '')} 모델을 사용해 계속 학습합니다.")
    else:
        total_epoch = 0
        print('저장된 가중치가 없어 처음부터 학습합니다.')

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    EPOCHS = 100
    mini_epoch = 10

    evaluate = Evaluate(model, tokenizer)
    for e in range(int(EPOCHS/mini_epoch)):
        model.fit(dataset, epochs=mini_epoch, workers=-1, callbacks=[early_stopping])
        total_epoch += mini_epoch
        with open(epoch_saved_path, 'w', encoding='utf-8') as f:
            f.write(str(total_epoch))
        print('>'*20, '샘플 예측', '<'*20)
        sample_sentence = "오늘 엄마랑 에버랜드 갔다왔어."
        evaluate.predict(sample_sentence)
        print('>'*50)

        if total_epoch % 5 == 0:
            model_path =  f'checkpoints/weights/transformer_weight_vocab_{tokenizer.vocab_size}_layers_{NUM_LAYERS}_{total_epoch}.h5'
            model.save_weights(model_path)

In [ ]:
tf.keras.backend.clear_session()
train_model(epochs=20)

In [ ]:
model.summary()